# This NB demonstrates how we can use methods from visualizations

We can use the method to plot all patterns, double click on specific patterns to highlight them in the heatmaps, and select top patterns to plot them on their own.

A html version of the notebook is available at https://trangdang168.github.io/CRIM_Intervals_Notebooks/Trang_CRIM_Intervals_Morgan.html. If this notebook doesn't work properly, please visit this link to interact with the visualizations.

In [1]:
# python imports
import pandas as pd
import altair as alt

# our imports
from main import *
import visualizations_demo as viz_demo

## `plot_ngrams_heatmap`

First, we do the normal steps to get a ngrams dataframe.

In [2]:
root = "https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_3.0/"
prefix = "CRIM_Model_00"
files = ["08"] 
postfix = ".mei"
corpus = CorpusBase([root + prefix + files[0] + postfix])
model = corpus.scores[0]
mel = model.getMelodic(kind='d', compound=False, unit=0)
mel_ngrams = model.getNgrams(df=mel, n=5)
mel_ngrams.head()

Successfully imported.


,[Superius],Altus,Tenor,Bassus
4.0,"4, 1, 2, 2, -3",NaN,NaN,NaN
20.0,NaN,"4, 1, 2, 2, -3",NaN,NaN
36.0,NaN,NaN,"4, 1, 2, 2, -3",NaN
52.0,NaN,NaN,NaN,"4, 1, 2, 2, -3"
62.0,"-2, -2, -2, 2, -2",NaN,NaN,NaN


Can read the documentation with `help(viz_demo.plot_ngrams_heatmap)`

In [3]:
help(viz_demo.plot_ngrams_heatmap)

Help on function plot_ngrams_heatmap in module visualizations_demo:

plot_ngrams_heatmap(ngrams_df, ngrams_duration=None, selected_patterns=[], voices=[], heatmap_width=800, heatmap_height=300)
    Plot a heatmap for crim-intervals getNgram's output.
    :param ngrams_df: crim-intervals getNgram's output
    :param ngrams_duration: if not None, rely on durations in the
    df to calculate the durations of the ngrams.
    :param selected_patterns: list of specific patterns the users want (optional)
    :param voices: list of specific voices the users want (optional)
    :param heatmap_width: the width of the final heatmap (optional)
    :param heatmap_height: the height of the final heatmap (optional)
    :return: a bar chart that displays the different patterns and their counts,
    and a heatmap with the start offsets of chosen voices / patterns



In [4]:
viz_demo.plot_ngrams_heatmap(mel_ngrams,selected_patterns=[], voices=[])

alt.VConcatChart(...)

### Selecting patterns

We could only include some patterns in the heatmaps

First, I collected the top patterns and turned these patterns into a list.

In [5]:
# count and get the 10 most popular patterns
mel_ngrams_top_patterns_df = mel_ngrams.stack().value_counts().to_frame().head(10)
# retrieve a list to generate heatmaps
mel_ngrams_top_patterns_list = mel_ngrams_top_patterns_df.index.to_list()
mel_ngrams_top_patterns_df

,0
"1, 2, -2, -2, -2",13
"2, -2, -2, -2, -2",11
"1, 1, 2, -2, -2",9
"-2, -2, -2, 2, 2",8
"2, -3, 2, -2, -2",8
"-3, 2, -2, -2, -2",8
"-2, -2, -2, -2, 2",7
"2, -3, 2, -3, 2",7
"-3, 2, 2, 2, -3",6
"2, -2, 1, -2, 2",6


In [6]:
mel_ngrams_top_patterns_list 

['1, 2, -2, -2, -2',
 '2, -2, -2, -2, -2',
 '1, 1, 2, -2, -2',
 '-2, -2, -2, 2, 2',
 '2, -3, 2, -2, -2',
 '-3, 2, -2, -2, -2',
 '-2, -2, -2, -2, 2',
 '2, -3, 2, -3, 2',
 '-3, 2, 2, 2, -3',
 '2, -2, 1, -2, 2']

We would pass the list of patterns into the `patterns` parametter.

In [7]:
viz_demo.plot_ngrams_heatmap(mel_ngrams, selected_patterns=mel_ngrams_top_patterns_list)

alt.VConcatChart(...)

**Cross-checking output from the heatmap with the ngrams dataframe**

We can see that '-2, -2, -2, -2, -2' is a really popular pattern. Let's view its start points in ngram and in the heatmap's dataframe to make sure that it has been calculated correctly

In [8]:
mel_ngrams[mel_ngrams ==  '-2, -2, -2, -2, -2' ].stack()

147.0  [Superius]    -2, -2, -2, -2, -2
161.0  [Superius]    -2, -2, -2, -2, -2
219.0  Tenor         -2, -2, -2, -2, -2
676.0  [Superius]    -2, -2, -2, -2, -2
dtype: object

In [9]:
viz_demo.plot_ngrams_heatmap(mel_ngrams, selected_patterns=['-2, -2, -2, -2, -2' ])

alt.VConcatChart(...)

## We can use `plot_ngrams_heatmap` with getDuration to create heatmap with durations!

How could we create a heatmap for ngrams in which the ngrams' durations are desplayed?

Let's look at the method of interest, `model.getDuration()`'s documentation.

In [35]:
help(model.getDuration)

Help on method getDuration in module main_objs:

getDuration(df=None, n=1, mask_df=None) method of main_objs.ImportedPiece instance
    If no arguments are passed, return a `pandas.DataFrame` of floats giving
    the duration of notes and rests in each part where 1 = quarternote,
    1.5 = a dotted quarter, 4 = a whole note, etc. If a df is passed, then
    return a df of the same shape giving the duration of each of the cells
    of this df. This is useful if you want to know what the durations of
    something other than single notes and rests, such as the durations of
    intervals. E.g.:
    
    har = importedPiece.getHarmonic()
    harDur = importedPiece.getDuration(df=har)
    
    The `n` parameter should be an integer greater than zero, or -1. When
    n is a positive integer, it groups together a sliding window of n
    consecutive non-NaN cells in each column. If you pass a df, it will sum
    the durations 'Rest' and non-Rest cell, provided they are in the same
    n-sized 

Because we created `mel_ngrams` with the code above:

```
mel = model.getMelodic(kind='d', compound=False, unit=0)
mel_ngrams = model.getNgrams(df=mel, n=5)
mel_ngrams.head()
```

We would use `df=mel`, `n=5`, and `mask_df=mel_ngrams` for `model.getDuration()`.

In [37]:
mel_ngrams_duration = model.getDuration(df=mel, n=5, mask_df=mel_ngrams)

In [38]:
viz_demo.plot_ngrams_heatmap(mel_ngrams, ngrams_duration, selected_patterns=mel_ngrams_top_patterns_list , voices=[])

alt.VConcatChart(...)

## `plot_close_match_heatmap`

Compute the Levenshtein distance between the two ngrams and display them on a heatmap.

You could real more about Levenshtein distance on [Wikipedia](https://en.wikipedia.org/wiki/Levenshtein_distance#:~:text=Informally%2C%20the%20Levenshtein%20distance%20between,considered%20this%20distance%20in%201965.).


In [13]:
help(viz_demo.plot_close_match_heatmap)

Help on function plot_close_match_heatmap in module visualizations_demo:

plot_close_match_heatmap(ngrams_df, key_pattern, ngrams_duration=None, selected_patterns=[], voices=[], heatmap_width=800, heatmap_height=300)
    Plot how closely the other vectors match a selected vector.
    Uses the Levenshtein distance.
    :param ngrams_df: crim-intervals getNgram's output
    :param key_pattern: a pattern the users selected to compare other patterns with (tuple of floats)
    :param selected_pattern: the specific other vectors the users selected
    :param ngrams_duration: if None, simply output the offsets. If the users input a
    list of durations, caculate the end by adding durations with offsets and
    display the end on the heatmap accordingly.
    :param selected_patterns: list of specific patterns the users want (optional)
    :param voices: list of specific voices the users want (optional)
    :param heatmap_width: the width of the final heatmap (optional)
    :param heatmap_heig

In [14]:
viz_demo.plot_close_match_heatmap(mel_ngrams, '-2, -2, -2, -2, -2', selected_patterns=[], voices=[])

alt.Chart(...)

**Additional Clarification**:
    
- As we change the cutoff with the slider, the ngrams with scores lower than the cutoff would be hidden away.
- Levenshtein distance takes into account how many changes we need to make to turn a pattern into another. For example, pattern '2, 2, 1, 2' and pattern '2, 2, 1, 3' would have the same levenshtein score 25 when compared to pattern '2, 2, 1, 1' because we need to make one change at the fourth character of the two strings to turn it into '2, 2, 1, 1'

In [15]:
# use the durations calculated from above
viz_demo.plot_close_match_heatmap(ngrams,'-2, -2, -2, -2, -2', ngrams_duration, selected_patterns=[], voices=[])

alt.Chart(...)

## `plot_comparison_heatmap`

This method can be used to plot a relationship/observation json.

Temporarily switch over to model 0009 because there's a problem in the ema's address for model 0008, so model 0008 can't be plotted properly.

### For relationships

In [16]:
df_relationships = pd.read_csv('crim_relationships.csv')
df_relationships_test = df_relationships[df_relationships['model_observation.piece.piece_id'] == 'CRIM_Model_0009'].copy()

**Plotting the same heatmap for other models/masses**:

You could refer to the list of available models/masses below and replace 'CRIM_Model_0009' with the piece of interest to plot the heatmap for that piece.

In [39]:
df_relationships['model_observation.piece.piece_id'].unique()

array(['CRIM_Model_0011', 'CRIM_Model_0018', 'CRIM_Model_0017',
       'CRIM_Mass_0015_2', 'CRIM_Model_0001', 'CRIM_Model_0020',
       'CRIM_Model_0009', 'CRIM_Model_0019', 'CRIM_Model_0010',
       'CRIM_Model_0022', 'CRIM_Model_0002', 'CRIM_Model_0016',
       'CRIM_Model_0024', 'CRIM_Model_0021', 'CRIM_Mass_0018_1',
       'CRIM_Model_0008', 'CRIM_Mass_0018_3', 'CRIM_Model_0012',
       'CRIM_Model_0023', 'CRIM_Model_0015', 'CRIM_Model_0013',
       'CRIM_Mass_0005_1', 'CRIM_Mass_0016_2', 'CRIM_Mass_0020_3',
       'CRIM_Model_0006', 'CRIM_Model_0025'], dtype=object)

In [17]:
df_relationships_test.head()

,Unnamed: 0,url,id,relationship_type,musical_type,rt_q,rt_q_x,rt_q_monnayage,rt_tm,rt_tm_snd,...,model_observation.piece.url,model_observation.piece.piece_id,model_observation.piece.full_title,model_observation.ema,derivative_observation.url,derivative_observation.id,derivative_observation.piece.url,derivative_observation.piece.piece_id,derivative_observation.piece.full_title,derivative_observation.ema
80,80,http://crimproject.org/data/relationships/81/,81,Quotation,Fuga,True,False,True,False,False,...,http://crimproject.org/data/pieces/CRIM_Model_...,CRIM_Model_0009,Je suis déshéritée,"1-5/3,3,1+3,1+3,1/@1-3,@1-4,@1-4+@1-3,@1-3+@1,@1",http://crimproject.org/data/observations/162/,162,http://crimproject.org/data/pieces/CRIM_Mass_0...,CRIM_Mass_0006_1,Missa Je suis déshéritée: Kyrie,"1-5/3,3,1+3,1,1/@1-3,@1-3,@1-4+@1-3,@1-3,@1"
81,81,http://crimproject.org/data/relationships/82/,82,Non-mechanical transformation,Fuga,False,False,False,False,False,...,http://crimproject.org/data/pieces/CRIM_Model_...,CRIM_Model_0009,Je suis déshéritée,"1-5/3-4,2-4,1-4,1-3,1/@3+@1-4,@1-4+@1-4+@1-4.5...",http://crimproject.org/data/observations/164/,164,http://crimproject.org/data/pieces/CRIM_Mass_0...,CRIM_Mass_0006_1,Missa Je suis déshéritée: Kyrie,"1-5/3-4,2-4,1-4,1-2+4,1/@3+@1-4,@1-4+@1-3+@1-4..."
107,107,http://crimproject.org/data/relationships/108/,108,Quotation,Soggetto,True,False,True,False,False,...,http://crimproject.org/data/pieces/CRIM_Model_...,CRIM_Model_0009,Je suis déshéritée,"1-3/3,3,3/@3,@1-4,@1",http://crimproject.org/data/observations/216/,216,http://crimproject.org/data/pieces/CRIM_Mass_0...,CRIM_Mass_0007_1,Missa Je suis déshéritée: Kyrie,"1-3/3,3,3/@3,@1-3,@1-3"
108,108,http://crimproject.org/data/relationships/109/,109,Non-mechanical transformation,Fuga,False,False,False,False,False,...,http://crimproject.org/data/pieces/CRIM_Model_...,CRIM_Model_0009,Je suis déshéritée,"1-5/3,3,1+3,1+3,1/@3,@1-4,@1-4+@1-3,@1-3+@1,@1",http://crimproject.org/data/observations/218/,218,http://crimproject.org/data/pieces/CRIM_Mass_0...,CRIM_Mass_0007_1,Missa Je suis déshéritée: Kyrie,"1-5/3,3,1+3,1,1/@3,@1-3,@3+@1-3,@1-3,@1-3"
109,109,http://crimproject.org/data/relationships/110/,110,Non-mechanical transformation,Fuga,False,False,False,False,False,...,http://crimproject.org/data/pieces/CRIM_Model_...,CRIM_Model_0009,Je suis déshéritée,"5-9/3,3,1+3,1+3,1/@2-4,@1-4,@2-4+@1-3,@1-4+@1,@1",http://crimproject.org/data/observations/220/,220,http://crimproject.org/data/pieces/CRIM_Mass_0...,CRIM_Mass_0007_1,Missa Je suis déshéritée: Kyrie,"5-9/4,1+4,1-2+4,2+4,4/@3,@2-4+@1-3,@1-2+@2-4+@..."


In [18]:
help(viz_demo.plot_comparison_heatmap)

Help on function plot_comparison_heatmap in module visualizations_demo:

plot_comparison_heatmap(df, ema_col, main_category='musical_type', other_category='observer.name', option=1, heat_map_width=800, heat_map_height=300)
    This method plots a chart for relationships/observations dataframe retrieved from their
    corresponding json files. This chart has two bar charts displaying the count of variables
    the users selected, and a heatmap displaying the locations of the relationship.
    :param df: relationships or observations dataframe
    :param ema_col: name of the ema column
    :param main_category: name of the main category for the first bar chart.
    The chart would be colored accordingly (default='musical_type').
    :param other_category: name of the other category for the zeroth bar chart.
    (default='observer.name')
    :param heat_map_width: the width of the final heatmap (default=800)
    :param heat_map_height: the height of the final heatmap (default =300)
    :r

In [19]:
viz_demo.plot_comparison_heatmap(df_relationships_test, 'model_observation.ema', main_category='musical_type', other_category='observer.name', heat_map_width=800,
                              heat_map_height=300)

alt.VConcatChart(...)

**We can command+click on the bars to open a new tab containing the information about the observations on the website**

### For observations

In [20]:
# data_observations = requests.get('http://crimproject.org/data/observations/').json()
# df_observations = pd.json_normalize(data_observations)
# df_observations.to_csv('crim_observations.csv')

In [21]:
df_observations = pd.read_csv('crim_observations.csv')
df_observations_test = df_observations[df_observations['piece.piece_id'] == 'CRIM_Model_0009'].copy()

In [22]:
viz_demo.plot_comparison_heatmap(df_observations_test, 'ema', main_category='musical_type', other_category='observer.name', heat_map_width=800,
                              heat_map_height=300)

alt.VConcatChart(...)

## Generate networks with `create_comparisons_networks_and_interactive_df`

We can create a some network visualizations and a df to interact with patterns.

Work with both time interval and melodic interval!

**Important**:
- The plot will be saved in a html file in the folder you choose.
- The plot might not be displayed if it's not saved in the same folder as the Jupyter Notebook, but you should be able to find it and open it in your computer if necessary.
- The `segments` column in the interactive df shows the range of the measures the observation was found, for example: `['5-9', '14-18']` means that the observation have been found from measure 5 to measure 9, and again from measure 14 to measure 18.

### Melodic interval for pens

The columns of interest from table containing observations fetched from the CRIM database would be `mt_pe_int` (`mt` = musical type, `pe` = periodic entries, `int`= intervals) and we would choose `interval_type` to be `melodic intervals`. The table's corresponding ema column is `ema`.

In [23]:
pen_networks, pen_widget = viz_demo.create_comparisons_networks_and_interactive_df(df_observations, 'mt_pe_int', 'melodic', 'ema')

interactive(children=(Text(value='Input search pattern', description='search_pattern_starts_with'), Output()),…

**See all the available starting intervals**:

- the 'all' key means that this networks contains all of the pens.
- If you select a network with specific starting intervals (e.g: 8+), it only contains pens that start with 8+.

In [24]:
pen_networks

{'all': <class 'pyvis.network.Network'> |N|=81 |E|=413,
 'nan': <class 'pyvis.network.Network'> |N|=0 |E|=0,
 '5+': <class 'pyvis.network.Network'> |N|=13 |E|=19,
 '8+': <class 'pyvis.network.Network'> |N|=10 |E|=14,
 '1+': <class 'pyvis.network.Network'> |N|=20 |E|=26,
 '2+': <class 'pyvis.network.Network'> |N|=2 |E|=3,
 '4-': <class 'pyvis.network.Network'> |N|=4 |E|=4,
 '9+': <class 'pyvis.network.Network'> |N|=2 |E|=1,
 '7+': <class 'pyvis.network.Network'> |N|=6 |E|=5,
 '8-': <class 'pyvis.network.Network'> |N|=6 |E|=65,
 '5-': <class 'pyvis.network.Network'> |N|=8 |E|=9,
 '4+': <class 'pyvis.network.Network'> |N|=6 |E|=18,
 '6-': <class 'pyvis.network.Network'> |N|=1 |E|=0,
 '9-': <class 'pyvis.network.Network'> |N|=2 |E|=1}

In [25]:
# show all the patterns mapped onto one network
pen_networks['all'].show('all_pen.html')

In [26]:
# show only one family of patterns mapped onto one network
pen_networks['8+'].show('8+_pen.html')

### melodic intervals for fuga

The columns of interest from table containing observations fetched from the CRIM database would be `mt_fg_int` (mt= musical type, fg= Fuga, int=intervals) and we would choose `interval_type` to be `melodic`. The table's corresponding ema column is `ema`.

In [27]:
fuga_networks, fuga_widget = viz_demo.create_comparisons_networks_and_interactive_df(df_observations, 'mt_fg_int', 'melodic', 'ema')

interactive(children=(Text(value='Input search pattern', description='search_pattern_starts_with'), Output()),…

In [28]:
# we are interested in the patterns that start with 4-
fuga_networks['4-'].show('4-_fuga.html')

In [29]:
fuga_networks

{'all': <class 'pyvis.network.Network'> |N|=905 |E|=2,715,
 '4-': <class 'pyvis.network.Network'> |N|=68 |E|=137,
 '8-': <class 'pyvis.network.Network'> |N|=108 |E|=184,
 '5-': <class 'pyvis.network.Network'> |N|=123 |E|=293,
 'nan': <class 'pyvis.network.Network'> |N|=0 |E|=0,
 '4+': <class 'pyvis.network.Network'> |N|=117 |E|=206,
 '1+': <class 'pyvis.network.Network'> |N|=112 |E|=203,
 '8+': <class 'pyvis.network.Network'> |N|=92 |E|=259,
 '5+': <class 'pyvis.network.Network'> |N|=128 |E|=255,
 '12+': <class 'pyvis.network.Network'> |N|=19 |E|=25,
 '2+': <class 'pyvis.network.Network'> |N|=30 |E|=86,
 '12-': <class 'pyvis.network.Network'> |N|=25 |E|=27,
 '15-': <class 'pyvis.network.Network'> |N|=1 |E|=0,
 '6+': <class 'pyvis.network.Network'> |N|=1 |E|=0,
 '15+': <class 'pyvis.network.Network'> |N|=1 |E|=0,
 '2-': <class 'pyvis.network.Network'> |N|=3 |E|=4,
 '9+': <class 'pyvis.network.Network'> |N|=7 |E|=7,
 '7-': <class 'pyvis.network.Network'> |N|=3 |E|=2,
 '9-': <class 'pyvis

## `plot_relationship_network`

This method accepts a dataframe of CRIM relationships and generate a network of observations.

In the network:

- the nodes are the observations (labeled with the piece and measures they are in)
- the edges are the relationships. 
    - points from a model_observation to a derivative observation.
    - are weighted differently based on the type of the relationships.
    - are labeled with the relationship type
    - are colored based on the users' choice.

### WARNING: Edge color isn't always a correct indicator of the relationship type.

The color of the edges is derived from the nodes; therefore, they would not be colored correctly when:

1. color='derivative'

model1 --(quotation)--> model2

model3 --(new material)--> model2

because of the first relationship, model2 and the two edges pointing to it would be colored with quotation's color.

2. color='model'

model1 --(quotation)--> model2

model1 --(new material)--> model3

because of the first relationship, model1 and the two out edges from model1 would be colored with quotation's color

In this case, we should pay attention to the edges' thickness (different weights mean different relationship types) or hover over the edges to see the relationship type labels.

### Seeing the big picture with the color modes being `model` or `derivative`

We have two options for the `color` parameter. 

- `derivative` (default): the edges and nodes of derivative observations will be colored according to the relationships. 
- `model` (default): the edges and nodes of model observations will be colored according to the relationships. 

**Here I use df_relationships.head(300) instead of df_relationships because plotting all of the relationships usually takes really long**.

#### color=`derivative`

In [30]:
nt = viz_demo.plot_relationship_network(df_relationships.head(300), color='derivative')
nt.show('small_relationships_network_derivative.html')

#### color=`model`

In [31]:
nt2 = viz_demo.plot_relationship_network(df_relationships.head(300), color='model')
nt2.show('small_relationships_network_model.html')

### Making uses of the other parameters.

The two plots are too cluttered so we can't see the relationship_type between specific measures inside pieces. 

#### Selecting specific models with `selected_model_ids` and specific derivatives with`selected_derivative_ids` 

In [32]:
nt3 = viz_demo.plot_relationship_network(df_relationships, selected_derivative_ids=['CRIM_Mass_0017_3', 'CRIM_Mass_0008_4',
       'CRIM_Mass_0003_5', 'CRIM_Mass_0014_4', 'CRIM_Mass_0018_1',
       'CRIM_Mass_0018_2', 'CRIM_Mass_0005_5'], color='derivative')
nt3.show('selected_derivatives.html')

In [33]:
nt4 = viz_demo.plot_relationship_network(df_relationships, selected_model_ids = ['CRIM_Model_0008', 'CRIM_Model_0016'], color='derivative')
nt4.show('selected_model.html')

#### Plotting pieces that are relevant to a list of interested pieces with `selected_members`
Having some of interest pieces, we want to plot all pieces that happen to connect to these pieces some how.

In [34]:
nt5 = viz_demo.plot_relationship_network(df_relationships, selected_model_ids = ['CRIM_Model_0008', 'CRIM_Model_0016'], selected_families=['CRIM_Mass_0017_3', 'CRIM_Mass_0008_4',
       'CRIM_Mass_0003_5', 'CRIM_Mass_0014_4', 'CRIM_Mass_0018_1',
       'CRIM_Mass_0018_2', 'CRIM_Mass_0005_5'], color='derivative')
nt5.show('selected_model_families.html')

5 CRIM_Mass_0017_3, CRIM_Mass_0008_4, CRIM_Mass_0003_5, CRIM_Mass_0018_1, CRIM_Mass_0018_2 no longer exist in df because of other filtering options
